<a href="https://colab.research.google.com/github/Korsner/nerv/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colaboratory!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка данных, подготовка данных и разбиение на выборки для обучения и тестирования.

In [6]:
import numpy as np
import keras
from keras import models
from keras import layers
from keras.datasets import reuters
from keras.utils import np_utils, to_categorical

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print("train_data ", train_data.shape)
print("train_labels ", train_labels.shape)
print("test_data ", test_data.shape)
print("test_labels ", test_labels.shape)

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in
train_data[0]])
print(decoded_newswire)
print(train_labels[0])

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
	
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
print("x_train ", x_train.shape)
print("x_test ", x_test.shape)

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)
print("one_hot_train_labels ", one_hot_train_labels.shape)
print("one_hot_test_labels ", one_hot_test_labels.shape)

x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]
print("x_val ", x_val.shape)
print("y_val ", y_val.shape)
print("partial_x_train ", partial_x_train.shape)
print("partial_y_train ", partial_y_train.shape)

train_data  (8982,)
train_labels  (8982,)
test_data  (2246,)
test_labels  (2246,)
? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3
3
x_train  (8982, 10000)
x_test  (2246, 10000)
one_hot_train_labels  (8982, 46)
one_hot_test_labels  (2246, 46)
x_val  (1000, 10000)
y_val  (1000, 46)
partial_x_train  (7982, 10000)
partial_y_train  (7982, 46)


Создание модели, слоёв, обучение, тестирование.

In [8]:
model = models.Sequential()
model.add(layers.Dense(1024, activity_regularizer=keras.regularizers.L2(0.01), activation='relu', input_shape=(10000,)))

model.add(layers.Dense(512, activity_regularizer=keras.regularizers.L2(0.01), activation='relu'))
model.add(layers.Dense(256, activity_regularizer=keras.regularizers.L2(0.01), activation='relu'))
model.add(layers.Dense(256, activity_regularizer=keras.regularizers.L2(0.01), activation='relu'))
model.add(layers.Dense(46, activity_regularizer=keras.regularizers.L2(0.01), activation='softmax'))

model.summary()
model.compile(optimizer = "RMSprop", loss = "categorical_crossentropy", metrics = ["accuracy"])
results = model.evaluate(x_val, y_val)

results = model.fit(partial_x_train, partial_y_train, epochs = 15, batch_size = 512, validation_data=(x_val, y_val))
score = model.evaluate(x_val, y_val)
print("Test точность: %.2f%%" % (score[1] * 100))


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1024)              10241024  
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 256)               65792     
                                                                 
 dense_9 (Dense)             (None, 46)                11822     
                                                                 
Total params: 10,974,766
Trainable params: 10,974,766
Non-trainable params: 0
_________________________________________________________________
32/32 [==============================] - 1